In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import sqlite3
import pandas as pd
import numpy as np

# Modell und Tokenizer laden
tokenizer = BertTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")
model = BertForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

In [7]:


# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect('derstandard.db')

# Liste der aktuell relevanten Parteien
current_parties = ['ÖVP', 'FPÖ', 'NEOS', 'Grüne', 'SPÖ', 'KPÖ']

# Erstellen einer formatgerechten Liste für SQL IN-Klausel
placeholders = ','.join(['?'] * len(current_parties))

# SQL-Abfrage zum Abrufen der Artikel, die eine der aktuellen Parteien behandeln
query = f'''
SELECT DISTINCT a.*
FROM Articles a
JOIN Article_Keywords ak ON a.articleID = ak.articleID
JOIN Keywords k ON ak.keywordID = k.keywordID
WHERE k.keyword IN ({placeholders})
'''

# Ausführen der Abfrage
df_articles = pd.read_sql_query(query, conn, params=current_parties)

# Umwandeln des Datums in ein datetime-Objekt
df_articles['datetime'] = pd.to_datetime(df_articles['datetime'])

# Nimm nur Artikel, die nach dem 1.1.2015 veröffentlicht wurden
df = df_articles[df_articles['datetime'] >= '2015-01-01']

# Verbindung schließen
conn.close()

# Einen Blick auf die geladenen Artikel werfen
df


,articleID,urlID,kicker,title,subtitle,author,datetime,article_text
346,43983,1262048,STANDARD-Umfrage,86 Prozent erwarten schwieriges Jahr für die R...,Nur der FPÖ wird ein erfolgreiches Jahr 2015 z...,Conrad Seidl,2015-01-01 17:22:00,Linz - Unter allen österreichischen Parteien t...
347,44110,1262176,Causa Hypo,Opposition macht mit Hypo-Ausschuss Ernst,"Beschluss in dritter Jänner-Woche, Grün granti...",Nina Weißensteiner,2015-01-02 16:40:00,"Wien - Kommende Woche, wenn der letzte Feierta..."
348,44156,1262222,RH-Bericht,Kammern verweigern Prüfung von Geschäften mit ...,Der Rechnungshof müsste die Geschäfte aller öf...,,2015-01-02 12:28:00,Wien – Der Rechnungshof beklagt die mangelnde ...
349,44179,1262245,Umfrage,"Faymann 2014 im Minus, Bonus für Mitterlehner",Turbulentes Vorjahr für die Parteichefs im APA...,,2015-01-02 10:28:00,Wien - Das turbulente innenpolitische Jahr 201...
350,44219,1262285,Reaktionen,Politiker äußern Betroffenheit über Kurt Kuchs...,"Darabos: ""Freund verloren"" - Blümel würdigt in...",,2015-01-03 19:11:00,Betroffen über den Tod des Journalisten Kurt K...
...,...,...,...,...,...,...,...,...
75893,504608,1791153,Ingrid Brodnig,Wahlkampf mit Klima? Muss sein!,"Wie abstrus es ist, jetzt eine große politisch...",Kolumne/Ingrid Brodnig,2024-09-19 07:58:00,Eine Aussage ärgert mich besonders: Die Meinun...
75894,504672,1791226,Nationalratswahl,Abtreibungen in Österreich: Sehen die Parteien...,Links stehende und liberale Parteien sind sich...,Beate Hausbichler,2024-09-18 13:53:00,Besonders in den vergangenen Monaten und Jahre...
75895,504797,1791376,Duell-Analyse,Kickl gegen Babler im TV-Duell: Fairplay ist n...,SPÖ-Chef Andreas Babler und FPÖ-Chef Herbert K...,Katharina MittelstaedtMichael Völker,2024-09-20 22:58:00,"Es ist Freitag, 21.35 Uhr, als der Wahlkampf s..."
75896,504926,1791561,Pressefreiheit,ÖVP-Entwurf: Justiz sollte bei Aktenzitaten in...,"Ein Regierungspapier, erarbeitet vom ÖVP-Klub,...",Fabian Schmid,2024-09-23 18:31:00,Die Pläne der ÖVP für ein Verbot von Verdachts...


In [8]:
# Funktion zur Berechnung der Tokenlänge eines Textes
def token_length(text):
    tokens = tokenizer(text, truncation=False, padding=False)
    return len(tokens['input_ids'])

# Anwendung der Funktion auf die 'article_text' Spalte
df['token_length'] = df['article_text'].apply(token_length)

# Überprüfen, wie viele Artikel länger als 512 Tokens sind
truncated_articles = df[df['token_length'] > 512]
print(f"Anzahl der Artikel, die länger als 512 Tokens sind: {len(truncated_articles)}")



Token indices sequence length is longer than the specified maximum sequence length for this model (562 > 512). Running this sequence through the model will result in indexing errors


Anzahl der Artikel, die länger als 512 Tokens sind: 48799


/var/folders/tb/9x7l6bn17ts_0wvhv5tr2lch0000gq/T/ipykernel_1574/879210033.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['token_length'] = df['article_text'].apply(token_length)


In [22]:
df['chunks_needed'] = df['token_length'].apply(lambda x: x // 512 + 1)

/var/folders/tb/9x7l6bn17ts_0wvhv5tr2lch0000gq/T/ipykernel_1574/244177996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chunks_needed'] = df['token_length'].apply(lambda x: x // 512 + 1)


In [23]:
df

,articleID,urlID,kicker,title,subtitle,author,datetime,article_text,token_length,chunks_needed
346,43983,1262048,STANDARD-Umfrage,86 Prozent erwarten schwieriges Jahr für die R...,Nur der FPÖ wird ein erfolgreiches Jahr 2015 z...,Conrad Seidl,2015-01-01 17:22:00,Linz - Unter allen österreichischen Parteien t...,307,1
347,44110,1262176,Causa Hypo,Opposition macht mit Hypo-Ausschuss Ernst,"Beschluss in dritter Jänner-Woche, Grün granti...",Nina Weißensteiner,2015-01-02 16:40:00,"Wien - Kommende Woche, wenn der letzte Feierta...",562,2
348,44156,1262222,RH-Bericht,Kammern verweigern Prüfung von Geschäften mit ...,Der Rechnungshof müsste die Geschäfte aller öf...,,2015-01-02 12:28:00,Wien – Der Rechnungshof beklagt die mangelnde ...,1246,3
349,44179,1262245,Umfrage,"Faymann 2014 im Minus, Bonus für Mitterlehner",Turbulentes Vorjahr für die Parteichefs im APA...,,2015-01-02 10:28:00,Wien - Das turbulente innenpolitische Jahr 201...,665,2
350,44219,1262285,Reaktionen,Politiker äußern Betroffenheit über Kurt Kuchs...,"Darabos: ""Freund verloren"" - Blümel würdigt in...",,2015-01-03 19:11:00,Betroffen über den Tod des Journalisten Kurt K...,760,2
...,...,...,...,...,...,...,...,...,...,...
75893,504608,1791153,Ingrid Brodnig,Wahlkampf mit Klima? Muss sein!,"Wie abstrus es ist, jetzt eine große politisch...",Kolumne/Ingrid Brodnig,2024-09-19 07:58:00,Eine Aussage ärgert mich besonders: Die Meinun...,717,2
75894,504672,1791226,Nationalratswahl,Abtreibungen in Österreich: Sehen die Parteien...,Links stehende und liberale Parteien sind sich...,Beate Hausbichler,2024-09-18 13:53:00,Besonders in den vergangenen Monaten und Jahre...,1469,3
75895,504797,1791376,Duell-Analyse,Kickl gegen Babler im TV-Duell: Fairplay ist n...,SPÖ-Chef Andreas Babler und FPÖ-Chef Herbert K...,Katharina MittelstaedtMichael Völker,2024-09-20 22:58:00,"Es ist Freitag, 21.35 Uhr, als der Wahlkampf s...",863,2
75896,504926,1791561,Pressefreiheit,ÖVP-Entwurf: Justiz sollte bei Aktenzitaten in...,"Ein Regierungspapier, erarbeitet vom ÖVP-Klub,...",Fabian Schmid,2024-09-23 18:31:00,Die Pläne der ÖVP für ein Verbot von Verdachts...,529,2


In [24]:
df['avg_chunkSize'] = df['token_length'] / df['chunks_needed']

df

/var/folders/tb/9x7l6bn17ts_0wvhv5tr2lch0000gq/T/ipykernel_1574/1547049999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_chunkSize'] = df['token_length'] / df['chunks_needed']


,articleID,urlID,kicker,title,subtitle,author,datetime,article_text,token_length,chunks_needed,avg_chunkSize
346,43983,1262048,STANDARD-Umfrage,86 Prozent erwarten schwieriges Jahr für die R...,Nur der FPÖ wird ein erfolgreiches Jahr 2015 z...,Conrad Seidl,2015-01-01 17:22:00,Linz - Unter allen österreichischen Parteien t...,307,1,307.000000
347,44110,1262176,Causa Hypo,Opposition macht mit Hypo-Ausschuss Ernst,"Beschluss in dritter Jänner-Woche, Grün granti...",Nina Weißensteiner,2015-01-02 16:40:00,"Wien - Kommende Woche, wenn der letzte Feierta...",562,2,281.000000
348,44156,1262222,RH-Bericht,Kammern verweigern Prüfung von Geschäften mit ...,Der Rechnungshof müsste die Geschäfte aller öf...,,2015-01-02 12:28:00,Wien – Der Rechnungshof beklagt die mangelnde ...,1246,3,415.333333
349,44179,1262245,Umfrage,"Faymann 2014 im Minus, Bonus für Mitterlehner",Turbulentes Vorjahr für die Parteichefs im APA...,,2015-01-02 10:28:00,Wien - Das turbulente innenpolitische Jahr 201...,665,2,332.500000
350,44219,1262285,Reaktionen,Politiker äußern Betroffenheit über Kurt Kuchs...,"Darabos: ""Freund verloren"" - Blümel würdigt in...",,2015-01-03 19:11:00,Betroffen über den Tod des Journalisten Kurt K...,760,2,380.000000
...,...,...,...,...,...,...,...,...,...,...,...
75893,504608,1791153,Ingrid Brodnig,Wahlkampf mit Klima? Muss sein!,"Wie abstrus es ist, jetzt eine große politisch...",Kolumne/Ingrid Brodnig,2024-09-19 07:58:00,Eine Aussage ärgert mich besonders: Die Meinun...,717,2,358.500000
75894,504672,1791226,Nationalratswahl,Abtreibungen in Österreich: Sehen die Parteien...,Links stehende und liberale Parteien sind sich...,Beate Hausbichler,2024-09-18 13:53:00,Besonders in den vergangenen Monaten und Jahre...,1469,3,489.666667
75895,504797,1791376,Duell-Analyse,Kickl gegen Babler im TV-Duell: Fairplay ist n...,SPÖ-Chef Andreas Babler und FPÖ-Chef Herbert K...,Katharina MittelstaedtMichael Völker,2024-09-20 22:58:00,"Es ist Freitag, 21.35 Uhr, als der Wahlkampf s...",863,2,431.500000
75896,504926,1791561,Pressefreiheit,ÖVP-Entwurf: Justiz sollte bei Aktenzitaten in...,"Ein Regierungspapier, erarbeitet vom ÖVP-Klub,...",Fabian Schmid,2024-09-23 18:31:00,Die Pläne der ÖVP für ein Verbot von Verdachts...,529,2,264.500000


In [9]:
# Funktion zur Sentiment-Analyse, unabhängig von der Textlänge
def analyze_sentiment(text, chunk_size=512):
    tokens = tokenizer(text, return_tensors="pt", truncation=False, padding=False)['input_ids'][0]
    
    # Text in Chunks von maximal 512 Tokens aufteilen (egal, wie lang der Text ist)
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    sentiments = []
    
    # Sentiment-Analyse für jeden Chunk durchführen
    for chunk in chunks:
        chunk = chunk.unsqueeze(0)
        outputs = model(input_ids=chunk)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits).item()
        sentiments.append(predicted_class_id)  # 0 = negativ, 1 = neutral, 2 = positiv
    
    # Durchschnittlichen Sentiment-Wert berechnen
    average_sentiment = np.mean(sentiments)
    
    # Klassifiziere den Durchschnitt in die drei Klassen
    if average_sentiment < 0.5:
        return 'negative'
    elif 0.5 <= average_sentiment < 1.5:
        return 'neutral'
    else:
        return 'positive'




In [21]:
df

,articleID,urlID,kicker,title,subtitle,author,datetime,article_text,token_length
346,43983,1262048,STANDARD-Umfrage,86 Prozent erwarten schwieriges Jahr für die R...,Nur der FPÖ wird ein erfolgreiches Jahr 2015 z...,Conrad Seidl,2015-01-01 17:22:00,Linz - Unter allen österreichischen Parteien t...,307
347,44110,1262176,Causa Hypo,Opposition macht mit Hypo-Ausschuss Ernst,"Beschluss in dritter Jänner-Woche, Grün granti...",Nina Weißensteiner,2015-01-02 16:40:00,"Wien - Kommende Woche, wenn der letzte Feierta...",562
348,44156,1262222,RH-Bericht,Kammern verweigern Prüfung von Geschäften mit ...,Der Rechnungshof müsste die Geschäfte aller öf...,,2015-01-02 12:28:00,Wien – Der Rechnungshof beklagt die mangelnde ...,1246
349,44179,1262245,Umfrage,"Faymann 2014 im Minus, Bonus für Mitterlehner",Turbulentes Vorjahr für die Parteichefs im APA...,,2015-01-02 10:28:00,Wien - Das turbulente innenpolitische Jahr 201...,665
350,44219,1262285,Reaktionen,Politiker äußern Betroffenheit über Kurt Kuchs...,"Darabos: ""Freund verloren"" - Blümel würdigt in...",,2015-01-03 19:11:00,Betroffen über den Tod des Journalisten Kurt K...,760
...,...,...,...,...,...,...,...,...,...
75893,504608,1791153,Ingrid Brodnig,Wahlkampf mit Klima? Muss sein!,"Wie abstrus es ist, jetzt eine große politisch...",Kolumne/Ingrid Brodnig,2024-09-19 07:58:00,Eine Aussage ärgert mich besonders: Die Meinun...,717
75894,504672,1791226,Nationalratswahl,Abtreibungen in Österreich: Sehen die Parteien...,Links stehende und liberale Parteien sind sich...,Beate Hausbichler,2024-09-18 13:53:00,Besonders in den vergangenen Monaten und Jahre...,1469
75895,504797,1791376,Duell-Analyse,Kickl gegen Babler im TV-Duell: Fairplay ist n...,SPÖ-Chef Andreas Babler und FPÖ-Chef Herbert K...,Katharina MittelstaedtMichael Völker,2024-09-20 22:58:00,"Es ist Freitag, 21.35 Uhr, als der Wahlkampf s...",863
75896,504926,1791561,Pressefreiheit,ÖVP-Entwurf: Justiz sollte bei Aktenzitaten in...,"Ein Regierungspapier, erarbeitet vom ÖVP-Klub,...",Fabian Schmid,2024-09-23 18:31:00,Die Pläne der ÖVP für ein Verbot von Verdachts...,529


In [10]:
df_100 = df.head(100)

df_100['sentiment'] = df_100['article_text'].apply(analyze_sentiment)

/var/folders/tb/9x7l6bn17ts_0wvhv5tr2lch0000gq/T/ipykernel_1574/130876107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100['sentiment'] = df_100['article_text'].apply(analyze_sentiment)


In [17]:
df['token_length'].describe()

count    75316.000000
mean       744.798316
std        493.712162
min         12.000000
25%        417.000000
50%        651.000000
75%        925.000000
max      14833.000000
Name: token_length, dtype: float64

In [19]:
# sort by 

df2 = df.sort_values(by='token_length').copy()

In [20]:
df2

,articleID,urlID,kicker,title,subtitle,author,datetime,article_text,token_length
70645,346382,1611034,Cremers Photoblog,Gernot Blümel eröffnet ein Wahlplakat,,,2020-09-29 14:46:00,"Wien, Lichtenfelsgasse, vor der ÖVP Zentrale",12
1383,74251,1293248,Cremers Photoblog,Strache und der Neue für Salzburg,,,2015-06-16 17:32:00,"Wien, Pressekonferenz im Parlamentsklub der FP...",14
67378,203527,1449080,Cremers Photoblog,Kurz auf Tour,,,2017-10-13 14:51:00,"Wien, Lichtenfelsgasse, ÖVP-Zentrale Link zum ...",14
15114,306775,1565998,Cremers Photoblog,15 Jahre Strache in Bildern,,,2019-10-03 17:39:00,Ein Rückblick auf Heinz-Christian Strache als ...,17
69804,318324,1578240,Cremers Photoblog,Im Winterpalais mit Sebastian Kurz,,,2020-01-06 17:07:00,"Wien, Himmelpfortgasse. Interviewtermin mit ÖV...",18
...,...,...,...,...,...,...,...,...,...
17500,347782,1612554,Wien-Wahl 2020,Der Fragebogen zur Wien-Wahl: Von Arbeitslosig...,"Für alle Unentschlossenen: elf Fragen, die zei...","Lara Hagen, Oona Kroisleitner, David Krutzler",2020-10-09 09:00:00,"Mehr Gurken statt Avocados, das Wahlrecht für ...",7915
22111,447229,1725581,Umbaupläne,"Chefredaktion kämpft für ""Wiener Zeitung"": ""Pr...","Werden die Regierungspläne umgesetzt, stehen d...",Interview/Oliver Mark,2023-03-21 09:00:00,Wien – Nach 320 Jahren könnte Schluss sein: Di...,8039
64728,81325,1305144,ORF-General,"""Problemdruck"" der ORF-Gebühren für Wrabetz no...","Alexander Wrabetz über die Generalswahl 2016, ...",Interview/Harald Fidler,2015-08-22 08:00:00,Wien – Wie nervös macht den ORF-General der Ge...,9292
13229,287417,1542644,Chronologie,Nur Einzelfälle? Die lange Liste rechter Ausru...,Seit Antritt der Regierung dokumentiert DER ST...,Feature,2019-04-23 07:40:00,"""Das ist doch nur ein Einzelfall"", heißt es se...",10498


In [13]:
df_100['sentiment'].value_counts()

sentiment
positive    89
neutral     11
Name: count, dtype: int64